In [2]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
import csv
import datetime
import time
import os
from selenium import webdriver

import matplotlib.pyplot as plt
import collections
from openpyxl import load_workbook

import math
import statistics
import numpy as np


In [14]:
def set_driver(url):
    
    ser = Service(DRIVER_PATH)
    options = webdriver.ChromeOptions()  # Set up Chinese
    # options.add_argument('--headless')
    options.add_argument('lang=zh_CN.UTF-8')  # Replacement of head
    options.add_argument(
        'User-Agent="Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36"')
    options.add_argument("--disable-infobars")
    options.add_argument("--headless")
    # PROXY = "11.456.448.110:8080"
    # options.add_argument('--proxy-server=%s' % PROXY)
    driver = webdriver.Chrome(
        service=ser,
        options=options)

    driver.get(url)
    print("finish setup driver")
    return driver

    
def init_filename(source, subject):
    # /呱吉/PTT.csv
    # /呱吉/FB.csv
    # ...
    folder = f"/{source}/"
    return subject + folder


def write_file():
    out_filename = ""
    with open(out_filename, 'a') as out_file:
        for line in in_file:

            out_file.write(parsed_line)


def init_csv(file_name):

    with open(file_name, 'w+') as out_file:
        write = csv.writer(out_file)
        header = ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']
        # PTT, 呱吉吃大便, http://, '2021-11-17', XXXX, XXXXX, 23
        write.writerow(header)


def string_to_datetime(s):
    return datetime.datetime.strptime(s, '%Y/%m/%d')

def generate_ytr_name_list(filename="Youtuber名單 - 道歉.csv"):
    ytr_list = []
    with open(filename, 'r') as f:
        rows = csv.reader(f)
        for row in rows:
            if row[1] == "":
                pass
            if row[1] == "頻道(官方帳號)":
                continue
            ytr_name = []
            ytr_name.append(row[1].split("\n")[0])

            ytr_name.extend(row[3].replace("、", "\n").split("\n"))
            ytr_list.append(list(dict.fromkeys(ytr_name)))
    return ytr_list


## constant

In [15]:
DRIVER_PATH = '/home/bobo/Desktop/bobo/ML_final/bobo/crawlers/chromedriver'
keywords_url = "https://www.ettoday.net/news_search/doSearch.php?keywords=館長"
tag_url = "https://www.ettoday.net/news/tag/統神/"

In [16]:
ytr_list = ["谷阿莫", "黃氏兄弟", "聖結石", "曾聖傑"]


In [18]:
ytr_list = generate_ytr_name_list("Youtuber名單 - 工作表3.csv")
ytr_list

[['阿滴英文', '阿滴', '都省瑞'],
 ['啾啾鞋'],
 ['亮生活 / Bright Side', '亮生活'],
 ['理科太太', '陳映彤'],
 ['Taiwan Bar', '台灣吧'],
 ['那些電影教我的事 Lessons from Movies'],
 ['志祺七七 X 圖文不符', '志祺'],
 ['柴鼠兄弟 ZRBros', '柴鼠兄弟'],
 ['許伯&簡芝—倉鼠人', '許伯&簡芝－倉鼠人', '許伯', '簡芝', '倉鼠人'],
 ['Cheap', 'cheap'],
 ['窮奢極慾'],
 ['三個字SunGuts'],
 ['維思維WeisWay', '維思維', 'WeisWay'],
 ['營養健身葛格Peeta', 'Peeta', '葛格'],
 ['蒼藍鴿的醫學天地', '蒼藍鴿', '蒼藍教主'],
 ['SKIMMY 你的網路閨蜜', 'skimmy你的網路閨密', 'skimmy'],
 ['劉芒'],
 ['Dr. Ivan 6', 'Dr.Ivan'],
 ['冏冏電台', '冏冏子', '冏星人', '冏冏'],
 ['超級歪SuperY', '超級歪'],
 ['快樂大學Happiness University', '快樂大學'],
 ['昆蟲擾西吳沁婕Dee the bugbuff', '昆蟲擾西吳沁婕', '昆蟲擾西', '吳沁婕'],
 ['好味營養師品瑄', '好味營養師', '品瑄', '品瑄營養師'],
 ['飽妮'],
 ['啟點文化'],
 ['英文易開罐'],
 ['講日文的台灣女生 Tiffany', '講日文的台灣女生Tiffany', '講日文的台灣女生'],
 ['胃酸人 위산맨', '胃酸人'],
 ['GARY G腿講NBA故事'],
 ['Onion Man', '洋蔥'],
 ['蛋哥超有事', '蛋哥'],
 ['微疼', '我得了一種吃菜就會死的病-微疼'],
 ['阿啾小劇場', '阿啾', '阿啾繪圖同萌'],
 ['鹿人與泥鰍小劇場', '鹿人', '泥鰍'],
 ['啾啾妹'],
 ['霸軒與小美 Baxuan & Mei', '霸軒'],
 ['DeluCat迪鹿', '迪鹿'],
 ['BugCat Capoo', 'BugCat', 'Ca

## tag

In [19]:
def scroll_to_button(driver):
    reached_page_end = False
    last_height = driver.execute_script("return document.body.scrollHeight")

    while not reached_page_end:
        driver.find_element(By.XPATH,'//body').send_keys(Keys.END)
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if last_height == new_height:
            reached_page_end = True
        else:
            last_height = new_height
    print("reach end")
    # driver.quit()
    
def crawl_page_tag(driver, filename):
    block_div = driver.find_element(By.CLASS_NAME, "c1")
    title_list = block_div.find_elements(By.XPATH,
        ".//div[@class='piece clearfix']")
    
    if title_list == []:
        print(title_list)
        print("is empty")
        return

    press = title_list[-1].find_element(By.TAG_NAME,
        "h3").find_element(By.TAG_NAME,"a")
    press.send_keys(Keys.NULL)

    print("len: ", len(title_list))

    filename = filename
    with open(filename, 'a') as f:
        write = csv.writer(f)

        header = ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']
        dic = {i: "" for i in header}

        for i in title_list:
            dic['Source'] = "ettoday"
            dic['Title'] = i.find_element(By.TAG_NAME,'h3').text.replace("  ", " ")
            dic['Link'] = i.find_element(By.TAG_NAME, 'a').get_attribute("href")
            dic['Date'] = i.find_element(By.CLASS_NAME,'date').text
            dic['Summary'] = i.find_element(By.CLASS_NAME,'summary').text
            dic['Text'] = ""
            dic['Like'] = 0

            line = dic.values()
            write.writerow(line)

In [20]:
# tag search
def crawl_tag(path, tag_url):
    driver = set_driver(tag_url)

    scroll_to_button(driver)

    time.sleep(10)

    crawl_page_tag(driver, filename=path+"ettoday_tag.csv")

    driver.quit()

## keywords

In [21]:
def crawl_page_keyword(driver, filename):
    block_div = driver.find_element(By.ID, "primary")
    box_list = block_div.find_elements(By.CLASS_NAME, "box_2")
    
    if box_list == []:
        print("is empty")
        return


    filename = filename
    with open(filename, 'a') as f:
        write = csv.writer(f)

        header = ['Source', 'Title', 'Link', 'Date', 'Summary', 'Text', 'Like']
        dic = {i: "" for i in header}

        for box in box_list:
            dic['Source'] = "ettoday"
            dic['Title'] = box.find_element(By.TAG_NAME, "h2").text.replace("\u3000", " ")
            dic['Link'] = box.find_element(By.TAG_NAME, 'a').get_attribute("href")
            dic['Date'] = box.find_element(By.CLASS_NAME, 'date').text.split("/ ")[1].replace(")", "")
            dic['Summary'] = box.find_element(By.CLASS_NAME, 'detail').text
            dic['Text'] = ""
            dic['Like'] = 0

            line = dic.values()
            write.writerow(line)


def check_exists_by_class(driver, class_name):
    try:
        driver.find_element(By.CLASS_NAME, class_name)
    except NoSuchElementException:
        return False
    return True

def get_max_page(driver):
    info = driver.find_element(By.CLASS_NAME, "info").text
    for t in ["第", "頁", "共", "頁"]:
        info = info.replace(t, "")
    info = info.replace(" ", "")
    current_page, max_page = map(int, info.split("|"))
    current_page, max_page
    return max_page


In [22]:
# search by keywords
def crawl_keyword(path, keywords_url):
    driver = set_driver(keywords_url)

    for i in range(get_max_page(driver)):
        print("page ", i)
        if i > 100:
            return 
        crawl_page_keyword(driver, filename=path+"ettoday_keywords.csv")
        time.sleep(10)
        
        try:
            next_page_button = driver.find_element(By.XPATH,"//*[text()=' > ']")
            driver.execute_script("arguments[0].click();", next_page_button)

        except Exception as e:
            print(e)
            current_url, page = driver.current_url.split("page=")
            driver.get(current_url + "page=" + str(int(page)+1));
            
            

    driver.quit()

## ytr crawling

In [23]:
with open("ytr_crawl.csv", 'w+') as f:
    for line in ytr_list:
        f.write(",".join(line))
        f.write('\n')

In [10]:
ytr_list = generate_ytr_name_list("Youtuber名單 - 工作表3.csv")
ytr_list

[['阿滴英文', '阿滴', '都省瑞'],
 ['啾啾鞋'],
 ['亮生活 / Bright Side', '亮生活'],
 ['理科太太', '陳映彤'],
 ['Taiwan Bar', '台灣吧'],
 ['那些電影教我的事 Lessons from Movies'],
 ['志祺七七 X 圖文不符', '志祺'],
 ['柴鼠兄弟 ZRBros', '柴鼠兄弟'],
 ['許伯&簡芝—倉鼠人', '許伯&簡芝－倉鼠人', '許伯', '簡芝', '倉鼠人'],
 ['Cheap', 'cheap'],
 ['窮奢極慾'],
 ['三個字SunGuts'],
 ['維思維WeisWay', '維思維', 'WeisWay'],
 ['營養健身葛格Peeta', 'Peeta', '葛格'],
 ['蒼藍鴿的醫學天地', '蒼藍鴿', '蒼藍教主'],
 ['SKIMMY 你的網路閨蜜', 'skimmy你的網路閨密', 'skimmy'],
 ['劉芒'],
 ['Dr. Ivan 6', 'Dr.Ivan'],
 ['冏冏電台', '冏冏子', '冏星人', '冏冏'],
 ['超級歪SuperY', '超級歪'],
 ['快樂大學Happiness University', '快樂大學'],
 ['昆蟲擾西吳沁婕Dee the bugbuff', '昆蟲擾西吳沁婕', '昆蟲擾西', '吳沁婕'],
 ['好味營養師品瑄', '好味營養師', '品瑄', '品瑄營養師'],
 ['飽妮'],
 ['啟點文化'],
 ['英文易開罐'],
 ['講日文的台灣女生 Tiffany', '講日文的台灣女生Tiffany', '講日文的台灣女生'],
 ['胃酸人 위산맨', '胃酸人'],
 ['GARY G腿講NBA故事'],
 ['Onion Man', '洋蔥'],
 ['蛋哥超有事', '蛋哥'],
 ['微疼', '我得了一種吃菜就會死的病-微疼'],
 ['阿啾小劇場', '阿啾', '阿啾繪圖同萌'],
 ['鹿人與泥鰍小劇場', '鹿人', '泥鰍'],
 ['啾啾妹'],
 ['霸軒與小美 Baxuan & Mei', '霸軒'],
 ['DeluCat迪鹿', '迪鹿'],
 ['BugCat Capoo', 'BugCat', 'Ca

In [24]:
os.path.exists("data/鍾明軒/ettoday_keywords.csv")

True

In [26]:
base_path = "data/"
if not os.path.exists(base_path):
    os.makedirs(base_path)

for ytr_name in ytr_list:
    
    if ytr_name == ['']:
        continue
        
    print(ytr_name)
    folder_path = base_path + ytr_name[0] + "/"
    
    # if folder not exists, create
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)    
    
    # if file exists, continue, else, create
    if os.path.exists(folder_path+"ettoday_tag.csv"):
        pass
    else:
        init_csv(folder_path+"ettoday_tag.csv")
        for name in ytr_name:
            keywords_url = "https://www.ettoday.net/news_search/doSearch.php?keywords=" + name
            try:
                crawl_keyword(folder_path, keywords_url)
            except Exception as e:
                print(e)
                continue       
        
    # if file exists, continue, else, create
    # if os.path.exists(folder_path+"ettoday_keywords.csv"):
    #     pass
    # else:
    init_csv(folder_path+"ettoday_keywords.csv")

    for name in ytr_name:
        tag_url = "https://www.ettoday.net/news/tag/" + name + "/"
        try:
            crawl_tag(folder_path, tag_url)
        except Exception as e:
            print(e)
            continue
        
    print("*"*20)
    print()

['阿滴英文', '阿滴', '都省瑞']
finish setup driver
reach end
len:  39
finish setup driver
reach end
len:  188
finish setup driver
reach end
len:  1
********************

['啾啾鞋']
finish setup driver
reach end
len:  15
********************

['亮生活 / Bright Side', '亮生活']
finish setup driver
reach end
[]
is empty
finish setup driver
reach end
[]
is empty
********************

['理科太太', '陳映彤']
finish setup driver
reach end
len:  200
finish setup driver
reach end
len:  14
********************

['Taiwan Bar', '台灣吧']
finish setup driver
reach end
len:  4
finish setup driver
reach end
len:  8
********************

['那些電影教我的事 Lessons from Movies']
finish setup driver
reach end
[]
is empty
********************

['志祺七七 X 圖文不符', '志祺']
finish setup driver
reach end
[]
is empty
finish setup driver
reach end
len:  3
********************

['柴鼠兄弟 ZRBros', '柴鼠兄弟']
finish setup driver
reach end
[]
is empty
finish setup driver
reach end
len:  13
********************

['許伯&簡芝—倉鼠人', '許伯&簡芝－倉鼠人', '許伯', '簡芝', '倉鼠人']
fini